In [1]:
import glob
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'  # 也可以使用 tensorflow
# os.environ['THEANO_FLAGS']='floatX=float32,device=cuda,exception_verbosity=high'
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cuda,optimizer=fast_compile'
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import keras.backend as K

K.set_image_data_format('channels_last')

import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, Input, LSTM, Concatenate, Dense
from keras.layers import Conv2DTranspose, Reshape, Activation, Cropping2D, Flatten, UpSampling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu
from keras.initializers import RandomNormal
from keras.datasets import mnist
from urllib.request import urlretrieve
from keras.optimizers import RMSprop, SGD, Adam
import matplotlib.pyplot as plt
import numpy as np
import cv2

# from read_data import *
#from load_data import DataLoader

%run /root/jupyter/inspace/sang-min/myo_proejct/load_data.py import DataLoader


'''
Model structure

Input : (100) Vector
Output : (64, 64, 1) image

Generator
100 -> 256
256 -> 16 x 16
16 x 16 -> 32 x 32
32 x 32 -> 64 x 64
64 x 64 -> 128 x 128

Discriminator
128 x 128 -> 64 x 64
64 x 64 -> 32 x 32
32 x 32 -> 16 x 16
16 x 16 -> 8 x 8
8 x 8 -> 64
64 -> 32 (16, 10)

'''
loader = DataLoader(data_path='./dataset_2018_05_16/')

conv_init = RandomNormal(0, 0.02)
gamma_init = RandomNormal(1., 0.02)


def generator_containing_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)

    return model


def generative_model(noise_size):
    # lstm_layer = LSTM(80, input_shape=lstm_size)(lstm_input)
    # _ = Concatenate(axis=-1)([lstm_layer, noise_input])
    print(" _ : ", noise_input)
    _ = Dense(256, input_shape=(100,), activation='relu', bias_initializer='glorot_normal',
              kernel_initializer='glorot_normal')(noise_input)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Reshape((16, 16, 1), input_shape=(256,))(_)

    _ = Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer=conv_init, input_shape=(16, 16, 1))(
        _)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=256, kernel_size=3, padding='same', kernel_initializer=conv_init,
                        input_shape=(16, 16, 128))(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)
    
    _ = UpSampling2D()(_)
    _ = Conv2D(filters=512, kernel_size=3, padding='same',  kernel_initializer=conv_init,
                        input_shape=(32, 32, 256))(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = UpSampling2D()(_)
    _ = Conv2D(filters=256, kernel_size=3, padding='same', kernel_initializer=conv_init,
                        input_shape=(64, 64, 512))(_)
    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = LeakyReLU(alpha=0.2)(_)

    #_ = UpSampling2D()(_)
    _ = Conv2D(filters=1, kernel_size=3, padding='same', kernel_initializer=conv_init,
               input_shape=(128, 128, 256))(_)
    _ = Activation(activation='tanh')(_)

    return Model(inputs=noise_input, outputs=_)


def discriminative_model(image_size, image_channel):
    _ = inputs = Input(shape=(image_size, image_size, image_channel))

    _ = Conv2D(filters=256, kernel_size=(2, 2), strides=2, padding='same', input_shape=(128, 128, 1),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=512, kernel_size=(1, 1), strides=2, padding='same', input_shape=(64, 64, 256),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=256, kernel_size=(2, 2), strides=2, padding='same', input_shape=(32, 32, 512),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=128, kernel_size=(2, 2), strides=2, padding='same', input_shape=(16, 16, 256),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=128, kernel_size=(2, 2), strides=2, padding='same', input_shape=(8, 8, 128),
               kernel_initializer=conv_init)(_)
    _ = LeakyReLU(alpha=0.2)(_)

    _ = BatchNormalization(axis=1, gamma_initializer=gamma_init)(_, training=1)
    _ = Conv2D(filters=1, kernel_size=(2, 2), strides=1, padding='same', input_shape=(4, 4, 128),
               kernel_initializer=conv_init)(_)
    # _ = LeakyReLU(alpha=0.2)(_)

    outputs = Flatten()(_)
    outputs = Dense(1, activation='sigmoid')(outputs)

    return Model(inputs=inputs, outputs=outputs)


lstm_size = (300, 16)
noise_size = 100
image_size = 128
input_size = 100
image_channel = 1
learning_rate = 2e-4
optimizer = Adam(0.0002, 0.5)

# lstm_input = Input(shape=lstm_size)
noise_input = Input(shape=(noise_size,))
real_image = Input(shape=(image_size, image_size, image_channel))

net_g = generative_model(noise_size)
net_g.summary()

net_d = discriminative_model(image_size, image_channel)
net_d.summary()

net_d.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

fake_image = net_g(noise_input)

net_d.trainable = False

valid = net_d(fake_image)

combined_model = Model(inputs=noise_input, outputs=valid)
combined_model.compile(loss='binary_crossentropy', optimizer=optimizer)

# net_g.compile(loss='binary_crossentropy', optimizer='SGD')


'''
loader = DataLoader(data_path='./MYO_Dataset_label/')

emg = loader.load_emg_data()
image, label = loader.load_image()

print(emg.shape)
print(image.shape, label)

emg = loader.get_emg_datas(10)
images, labels = loader.get_images(10)
print(emg.shape, images.shape, labels.shape)

'''

epoch = 15000
i = 0
time_0 = time.time()
err_d = err_g = 0
err_d_sum = 0
err_g_sum = 0
batch_size = 64

d_label = np.zeros(shape=[batch_size])
g_label = np.zeros(shape=[batch_size])

print(d_label)
print(d_label.shape)

while i < epoch:
    # x_train = loader.get_emg_datas(batch_size)
    images = loader.get_images(batch_size)
    noise = np.random.normal(size=(batch_size, noise_size))

    gan_image = net_g.predict(noise)
    print("gan imaga1 : ", gan_image.shape)

    d_loss_real = net_d.train_on_batch(images, np.ones(shape=(batch_size, 1)))
    d_loss_fake = net_d.train_on_batch(gan_image, np.zeros(shape=(batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # print("epoch[%d] d_loss[%f]" % (i, d_loss))

    g_loss = combined_model.train_on_batch(noise, np.ones(shape=(batch_size, 1)))

    # print("epoch[%d] g_loss[%f]" % (i, g_loss))

    print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (i, d_loss[0], 100 * d_loss[1], g_loss))

    if i % 500 == 0:
        gan_image = net_g.predict(noise)
        print("gan imaga2 : ", gan_image[0].shape)
        cv2.imwrite('./output_image2/' + 'fake_image'+ str(i) + '.png', gan_image[0] * 127.5)
        #cv2.imwrite('./output_image3/' + 'real_imadge'+ str(i) + '.png', images[0] * 127.5)

    i += 1;

net_g.save_weights("./output_image2/g_model.h5")
net_d.save_weights("./output_image2/d_model.h5")
combined_model.save_weights("./output_image2/combined_model.h5")

g_model_json = net_g.to_json()
with open("./output_image2/g_model.json", "w") as json_file:
    json_file.write(g_model_json)
    
d_model_json = net_d.to_json()
with open("./output_image2/d_model.json", "w") as json_file:
    json_file.write(d_model_json)

combined_model_json = combined_model.to_json()
with open("./output_image2/combined_model.json", "w") as json_file:
    json_file.write(combined_model_json)

print("Saved model to disk")

plt.figure(1, figsize=(16, 8))
plt.plot(d_loss_history)
plt.ylabel('d_loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./output_image2/d_loss_history.png')

#plt.show()

plt.figure(2, figsize=(16, 8))
plt.plot(g_loss_history)
plt.ylabel('g_loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./output_image2/g_loss_history.png')

#plt.show()\

plt.figure(3, figsize=(16, 8))
plt.plot(d_acc_history)
plt.ylabel('d_acc')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')

plt.savefig('./output_image2/d_acc_history.png')

plt.show()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


30
./dataset_2018_05_16/1/
15
 _ :  Tensor("input_1:0", shape=(?, 100), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       1280      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 128)       64        
_______________________________________________________

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.859038, acc.: 25.00%] [G loss: 0.664108]
gan imaga2 :  (128, 128, 1)
gan imaga1 :  (64, 128, 128, 1)
1 [D loss: 0.524332, acc.: 50.00%] [G loss: 0.807866]
gan imaga1 :  (64, 128, 128, 1)
2 [D loss: 0.421411, acc.: 100.00%] [G loss: 1.009788]
gan imaga1 :  (64, 128, 128, 1)
3 [D loss: 0.333614, acc.: 100.00%] [G loss: 1.162686]
gan imaga1 :  (64, 128, 128, 1)
4 [D loss: 0.309112, acc.: 100.00%] [G loss: 1.285450]
gan imaga1 :  (64, 128, 128, 1)
5 [D loss: 0.229538, acc.: 100.00%] [G loss: 1.406134]
gan imaga1 :  (64, 128, 128, 1)
6 [D loss: 0.193694, acc.: 100.00%] [G loss: 1.502871]
gan imaga1 :  (64, 128, 128, 1)
7 [D loss: 0.176579, acc.: 100.00%] [G loss: 1.544387]
gan imaga1 :  (64, 128, 128, 1)
8 [D loss: 0.177670, acc.: 100.00%] [G loss: 1.603683]
gan imaga1 :  (64, 128, 128, 1)
9 [D loss: 0.146373, acc.: 100.00%] [G loss: 1.694258]
gan imaga1 :  (64, 128, 128, 1)
10 [D loss: 0.122324, acc.: 100.00%] [G loss: 1.657318]
gan imaga1 :  (64, 128, 128, 1)
11 [D loss: 0.15

gan imaga1 :  (64, 128, 128, 1)
92 [D loss: 0.011811, acc.: 100.00%] [G loss: 4.049205]
gan imaga1 :  (64, 128, 128, 1)
93 [D loss: 0.012151, acc.: 100.00%] [G loss: 4.081526]
gan imaga1 :  (64, 128, 128, 1)
94 [D loss: 0.010695, acc.: 100.00%] [G loss: 4.100500]
gan imaga1 :  (64, 128, 128, 1)
95 [D loss: 0.009219, acc.: 100.00%] [G loss: 4.141092]
gan imaga1 :  (64, 128, 128, 1)
96 [D loss: 0.010110, acc.: 100.00%] [G loss: 4.124223]
gan imaga1 :  (64, 128, 128, 1)
97 [D loss: 0.010537, acc.: 100.00%] [G loss: 4.080637]
./dataset_2018_05_16/8/
gan imaga1 :  (64, 128, 128, 1)
98 [D loss: 0.025835, acc.: 100.00%] [G loss: 4.111658]
gan imaga1 :  (64, 128, 128, 1)
99 [D loss: 0.015018, acc.: 100.00%] [G loss: 4.021647]
gan imaga1 :  (64, 128, 128, 1)
100 [D loss: 0.017231, acc.: 100.00%] [G loss: 4.166965]
gan imaga1 :  (64, 128, 128, 1)
101 [D loss: 0.008665, acc.: 100.00%] [G loss: 4.531326]
gan imaga1 :  (64, 128, 128, 1)
102 [D loss: 0.008184, acc.: 100.00%] [G loss: 4.662981]
gan i

...

gan imaga1 :  (64, 128, 128, 1)
2502 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.089972]
gan imaga1 :  (64, 128, 128, 1)
2503 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.108774]
gan imaga1 :  (64, 128, 128, 1)
2504 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.085997]
gan imaga1 :  (64, 128, 128, 1)
2505 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.096048]
gan imaga1 :  (64, 128, 128, 1)
2506 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.059937]
gan imaga1 :  (64, 128, 128, 1)
2507 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.063686]
gan imaga1 :  (64, 128, 128, 1)
2508 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.118490]
gan imaga1 :  (64, 128, 128, 1)
2509 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.115773]
gan imaga1 :  (64, 128, 128, 1)
2510 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.088696]
gan imaga1 :  (64, 128, 128, 1)
2511 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.074536]
gan imaga1 :  (64, 128, 128, 1)
2512 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.029873]

gan imaga1 :  (64, 128, 128, 1)
2591 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.207850]
gan imaga1 :  (64, 128, 128, 1)
2592 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.228691]
gan imaga1 :  (64, 128, 128, 1)
2593 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.211234]
gan imaga1 :  (64, 128, 128, 1)
2594 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.185158]
gan imaga1 :  (64, 128, 128, 1)
2595 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.213245]
gan imaga1 :  (64, 128, 128, 1)
2596 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.218182]
gan imaga1 :  (64, 128, 128, 1)
2597 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.254093]
gan imaga1 :  (64, 128, 128, 1)
2598 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.228483]
./dataset_2018_05_16/6/
gan imaga1 :  (64, 128, 128, 1)
2599 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.238537]
gan imaga1 :  (64, 128, 128, 1)
2600 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.210569]
gan imaga1 :  (64, 128, 128, 1)
2601 [D loss: 0.000001, acc.: 100.

gan imaga1 :  (64, 128, 128, 1)
2680 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.275889]
gan imaga1 :  (64, 128, 128, 1)
2681 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.272976]
gan imaga1 :  (64, 128, 128, 1)
2682 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.289448]
./dataset_2018_05_16/12/
gan imaga1 :  (64, 128, 128, 1)
2683 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.288212]
gan imaga1 :  (64, 128, 128, 1)
2684 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.277599]
gan imaga1 :  (64, 128, 128, 1)
2685 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.275015]
gan imaga1 :  (64, 128, 128, 1)
2686 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.272913]
gan imaga1 :  (64, 128, 128, 1)
2687 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.308393]
gan imaga1 :  (64, 128, 128, 1)
2688 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.310619]
gan imaga1 :  (64, 128, 128, 1)
2689 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.291332]
gan imaga1 :  (64, 128, 128, 1)
2690 [D loss: 0.000001, acc.: 100

2768 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.438862]
gan imaga1 :  (64, 128, 128, 1)
2769 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.402083]
gan imaga1 :  (64, 128, 128, 1)
2770 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.409006]
gan imaga1 :  (64, 128, 128, 1)
2771 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.384764]
gan imaga1 :  (64, 128, 128, 1)
2772 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.381758]
gan imaga1 :  (64, 128, 128, 1)
2773 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.400253]
gan imaga1 :  (64, 128, 128, 1)
2774 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.448226]
gan imaga1 :  (64, 128, 128, 1)
2775 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.425301]
gan imaga1 :  (64, 128, 128, 1)
2776 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.430696]
gan imaga1 :  (64, 128, 128, 1)
2777 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.412148]
gan imaga1 :  (64, 128, 128, 1)
2778 [D loss: 0.000001, acc.: 100.00%] [G loss: 13.471180]
gan imaga1 :  (64, 128, 128, 1)

KeyboardInterrupt: 